# Bài tập về nhà Decision Tree, Random Forest


- Thực hiện các yêu cầu dưới đây với tập dữ liệu Australian credit
- Down dữ liệu tại đường link dưới đây và đặt vào folder /data

  https://archive.ics.uci.edu/ml/datasets/Statlog+(Australian+Credit+Approval)
    

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold



## 1. Tìm hiểu dữ liệu

In [ ]:
'''
- Tìm hiểu cấu trúc data tại trang web UCI
- Down các files, đặt trong folder data/
- Chú ý:
    - Đọc mô tả dữ liệu trong file 'australian.doc'
    - Kiểm tra DecisionTree/RandomForest của Sklearn có hỗ trợ với kiểu dữ liệu được cung cấp không
'''
data_path = '/content/australian.dat'
df = pd.read_csv(data_path, sep=' ', header=None)
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8102005)

# Do dữ liệu đa số là dạng chữ nên đa số hệ số tương quan sẽ khá thấp nên không dùng những mô hình tuyến tính
# Vậy ta dùng decision tree hoặc random forest
# Do dữ liệu này cân bằng nên ta chọn độ đo là accuracy

## 2. Decision Tree

### 2.1. Khảo sát các giá trị khác nhau của max_depth

In [ ]:
# Cách 1: Sử dụng hold out để khảo sát
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_valid, X_testt, y_valid, y_testt = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# 2. Tìm kiếm siêu tham số
best_accuracy = 0
best_max_depth = None
S = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for max_depth in S:
    # Huấn luyện mô hình với siêu tham số max_depth
    model = DecisionTreeClassifier(max_depth=max_depth)
    model.fit(X_train, y_train)

    # Đánh giá trên tập D_valid
    y_valid_pred = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_valid_pred)

    # Lưu lại siêu tham số nếu hiệu suất tốt hơn
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_max_depth = max_depth

print(f'Best max_depth: {best_max_depth} with accuracy: {best_accuracy}')

# 3. Đánh giá trên tập D_test với siêu tham số tốt nhất
best_model = DecisionTreeClassifier(max_depth=best_max_depth)
best_model.fit(X_train, y_train)
y_test_pred = best_model.predict(X_testt)
test_accuracy = accuracy_score(y_testt, y_test_pred)

print(f'Test accuracy with best max_depth ({best_max_depth}): {test_accuracy}')



Best max_depth: 4 with accuracy: 0.8543689320388349
Test accuracy with best max_depth (4): 0.8942307692307693


In [ ]:
# Cách 2: Dùng GridSearch
param_grid = {'max_depth': [4,5, 6, 7, 8]}

grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)  # Fit the GridSearchCV object to the training data
print(grid_search.best_params_)  # Now you can access the best parameters

{'max_depth': 4}


### 2.2. Tìm kiếm siêu tham số

In [ ]:
# Cách 1: Dùng holdout để khảo sát
param_candidates = {'max_depth':[5, 6, 7, 8], 'min_samples_split': [2, 3, 4], 'min_samples_leaf':[2, 3, 4]}
best_accuracy = 0
best_params = {}
for max_depth in param_candidates['max_depth']:
  for min_samples_split in param_candidates['min_samples_split']:
    for min_samples_leaf in param_candidates['min_samples_leaf']:
      model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
      model.fit(X_train, y_train)
      # Đánh giá trên tập D_valid
      y_valid_pred = model.predict(X_valid)
      accuracy = accuracy_score(y_valid, y_valid_pred)

      # Lưu lại siêu tham số nếu hiệu suất tốt hơn
      if accuracy > best_accuracy:
          best_accuracy = accuracy
          best_params = {'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
print(f"Best parameters: {best_params} with validation accuracy: {best_accuracy}")

# 3. Đánh giá trên tập D_test với siêu tham số tốt nhất
best_model = DecisionTreeClassifier(**best_params)
best_model.fit(X_train, y_train)
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Test accuracy with best parameters: {test_accuracy}")



Best parameters: {'max_depth': 7, 'min_samples_split': 2, 'min_samples_leaf': 2} with validation accuracy: 0.8640776699029126
Test accuracy with best parameters: 0.8309178743961353


In [ ]:
# Cách 2: Dùng GridSearch
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

# Khởi tạo GridSearchCV
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Huấn luyện mô hình
grid_search.fit(X_train, y_train)

# Kết quả tốt nhất
print(f"Best parameters found: {grid_search.best_params_}")
print(f"Best cross-validation accuracy: {grid_search.best_score_}")

# Đánh giá mô hình trên tập kiểm tra
best_dt = grid_search.best_estimator_
accuracy = best_dt.score(X_test, y_test)
print(f"Test accuracy with best parameters: {accuracy}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters found: {'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2}
Best cross-validation accuracy: 0.845978705978706
Test accuracy with best parameters: 0.782608695652174


## 3. Random Forest

### 3.1 Khảo sát các giá trị khác nhau của n_estimators

In [ ]:
# Cách 1: Dùng holdout để khảo sát
best_accuracy = 0
best_n_estimators = None
n_estimators_candidates = [10, 20, 30, 40, 50]
for n_estimators in n_estimators_candidates:
  model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
  model.fit(X_train, y_train)
  # Đánh giá trên tập D_valid
  y_valid_pred = model.predict(X_valid)
  accuracy = accuracy_score(y_valid, y_valid_pred)

  # Lưu lại siêu tham số nếu hiệu suất tốt hơn
  if accuracy > best_accuracy:
      best_accuracy = accuracy
      best_n_estimators = n_estimators

print(f'Best n_estimators: {best_n_estimators} with accuracy: {best_accuracy}')

# 3. Đánh giá trên tập D_test với siêu tham số tốt nhất
best_model = RandomForestClassifier(n_estimators=best_n_estimators, random_state=42)
best_model.fit(X_train, y_train)
y_test_pred = best_model.predict(X_testt)
test_accuracy = accuracy_score(y_testt, y_test_pred)

print(f'Test accuracy with best n_estimators ({best_n_estimators}): {test_accuracy}')
print(f'Best n_estimators: {best_n_estimators} with accuracy: {best_accuracy}')


Best n_estimators: 10 with accuracy: 0.8446601941747572
Test accuracy with best n_estimators (10): 0.875
Best n_estimators: 10 with accuracy: 0.8446601941747572


In [ ]:
# Cách 2: Dùng GridSearch

param_grid = {'n_estimators': [10, 20, 30, 40, 50]}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'n_estimators': 20}


### 3.2 Tìm kiếm siêu tham số

In [ ]:
# Cách 1: Dùng holdout validation để khảo sát
param = {'n_estimators' : [30, 100], 'criterion' : ['gini', 'entropy', 'log_loss'],
   'max_features': ['sqrt', 'log2']}
best_accuracy = 0
best_params = {}
for n_estimators in param['n_estimators']:
  for criterion in param['criterion']:
    for max_features in param['max_features']:
      model = RandomForestClassifier(n_estimators=n_estimators, criterion=criterion, max_features=max_features, random_state = 42)
      model.fit(X_train, y_train)
      # Đánh giá trên tập D_valid
      y_valid_pred = model.predict(X_valid)
      accuracy = accuracy_score(y_valid, y_valid_pred)

      if accuracy > best_accuracy:
          best_accuracy = accuracy
          best_params = {'n_estimators': n_estimators, 'criterion': criterion, 'max_features': max_features}

print(f"Best parameters: {best_params} with validation accuracy: {best_accuracy}")

best_model = RandomForestClassifier(**best_params, random_state = 42)
best_model.fit(X_train, y_train)
y_test_pred = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Test accuracy with best parameters: {test_accuracy}")





Best parameters: {'n_estimators': 30, 'criterion': 'entropy', 'max_features': 'sqrt'} with validation accuracy: 0.883495145631068
Test accuracy with best parameters: 0.8743961352657005


In [ ]:
# Cách 2: Dùng GridSearch

param_grid = [
   {'n_estimators' : [30, 100], 'criterion' : ['gini', 'entropy', 'log_loss'],
   'max_features': ['sqrt', 'log2'], 'random_state': [42]}
]
grid_search = GridSearchCV(RandomForestClassifier(), param_grid = param_grid, cv = 4, scoring = "accuracy", n_jobs = 2, verbose = 1)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)


Fitting 4 folds for each of 12 candidates, totalling 48 fits
{'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 100, 'random_state': 42}
